## Runs DE on different clusters. Also runs DE within each cluster for each condition.
Does this over each donor and combined

In [1]:
# Input info
se_f = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/SE.rds"
sample_names = "preB,postB"
outdir = "/data/Mito_Trace/output/pipeline/DUPI_april08_2021/MTblacklist_mtasnucl_Bonly/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_hg38_1_2/annotation_clones/de_btwnclust_ATAC/minPct_0_logfc0.1"

# Parameters
assay="ATAC"
minPct = 0.00
logfcthresh = 0.1
top_de = 3
p_thresh=0.1
test.use="LR"
latent.vars = "peak_region_fragments"

# Parallel processing
cores = 4

# R utils script
utils_script <- "/data/Mito_Trace/R_scripts/seurat_utils.R"



In [2]:
source(utils_script)
outdir = outdir
dir.create(outdir, showWarnings = FALSE)


Attaching SeuratObject



In [3]:
sample_names <- unlist(strsplit(sample_names, ",")[[1]])
sample_names

[1] "preB"  "postB"

In [4]:
library(GenomicRanges)
library(Seurat)
library(Signac)
library(GenomeInfoDb)
library(EnsDb.Hsapiens.v75)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(data.table)
library(magrittr)
library(cowplot)
library(metap)
library("RColorBrewer")

library(future)
plan()

plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomeInfoDb

Loading required package: ensembldb

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

Lo

sequential:
- args: function (..., envir = parent.frame())
- tweaked: FALSE
- call: NULL

Warning message:
"Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'."


In [5]:
se <- readRDS(se_f)#file.path(indir, paste0("allSamples.integrated.rds")))
DefaultAssay(se) <- assay
se

An object of class Seurat 
61893 features across 4112 samples within 2 assays 
Active assay: ATAC (42318 features, 0 variable features)
 1 other assay present: RNA
 2 dimensional reductions calculated: integrated_lsi, umap

## Run DE for  cluster vs all others 

In [6]:
btwnClust.outdir <- file.path(outdir, "btwnClust")
dir.create(btwnClust.outdir)


In [7]:
curr.outdir <- btwnClust.outdir

cluster.ids <- sort(unique(se$seurat_clusters))
Idents(se) <- se$seurat_clusters

all.pvals <- data.frame(matrix(nrow=dim(se)[1],ncol=0),
                          row.names=row.names(se))
all.names.sig <- c()
for (id1 in cluster.ids){ #or however many clusters you have
    try({
        print(paste("cluster", id1))
        response <- FindMarkers(
                      object = se,
                      ident.1 = id1, #"CD4 Naive",
                      min.pct = minPct, logfc.thresh=logfcthresh,
                      test.use = test.use, latent.vars = latent.vars
                    )
        response$p_val_adj_BH <- stats::p.adjust(response$p_val, method = "BH", n = length(response$p_val)) 
        response <- response %>% dplyr::arrange(p_val_adj_BH)
        print(head(response))
        #write.csv(response, file=file.path(clust_outdir, paste0("cluster_",i,"vsAll.DE.csv")))
        curr.sig <- response %>% dplyr::filter(p_val_adj_BH<p_thresh)
        names.sig <- rownames(curr.sig)
        print("names.sig")
        print(names.sig)
        #se <- subset(se, seurat_clusters == id1)

        
        if (assay == "ATAC"){
            print("ATAC")
            response.features <- ClosestFeature(se, regions = rownames(response))
            response.features = response.features %>% dplyr::mutate(gene.id = paste(gene_name,type, sep="_"))
            response$gene.id = response.features$gene.id
            feature.names.sig <- response[names.sig, "gene.id"]
        }else{
            feature.names.sig <- names.sig
        }
        write.csv(response,
                  file=file.path(curr.outdir, paste0("cluster",id1,".DE.csv")), quote=F)
        
        de.plots(se, names.sig, curr.outdir, curr.name=id1, max.size=10, to.heat=T,to.vln=T, 
                 feature.names=feature.names.sig)
        print(paste('saved cluster', id1))
        
        # Add to all pvals
        all.names.sig <- c(all.names.sig, head(names.sig, top_de))
        all.pvals[names.sig,id1] <- curr.sig$p_val_adj_BH

    })
}


all.names.sig <- unique(all.names.sig)
de.plots(se, all.names.sig, curr.outdir, curr.name="allDonors.allClusters", max.size=15)

dot <- DotPlot(se, scale=FALSE,
               features = head(all.names.sig,20), 
               cluster.idents=T) + RotatedAxis()
dot
ggsave(file.path(curr.outdir, "allClusters.top3de.png"))
ggsave(file.path(curr.outdir, "allClusters.top3de.pdf"))

## b) averaged top p-values (only average ones that were DE)
sig.all.ordered <- sort(rowMeans(-log10(all.pvals),na.rm=T), decreasing=T)
dot <- DotPlot(se, scale=F,
               features = names(head(sig.all.ordered,20)), 
               cluster.idents=T) + RotatedAxis()
dot
ggsave(file.path(curr.outdir, "allClusters.pvalsOrdered.scaleF.png"))
ggsave(file.path(curr.outdir, "allClusters.pvalsOrdered.scaleF.pdf"))
#de.plots(se, names(sig.all.ordered), outdir, curr.name="allDonors.allClusters.pvalsOrdered", max.size=20)



all.pvals.nafill <- all.pvals
all.pvals.nafill[,] <- -log10(zoo::na.fill(all.pvals,1))

sig.all.ordered <- sort(rowMeans(all.pvals.nafill), decreasing=T)
dot <- DotPlot(se, scale=F,
               features = head(names(sig.all.ordered),20), 
               cluster.idents=F) + RotatedAxis() + ylab("Cluster")
ggsave(file.path(btwnClust.outdir,
                 "allClusters.ovalsOrderedNA.scaleF.png"))
ggsave(file.path(btwnClust.outdir,
                 "allClusters.ovalsOrderedNA.scaleF.pdf"))
dot

[1] "cluster 0"
                               p_val avg_log2FC pct.1 pct.2    p_val_adj
chr17-82126354-82128111 9.520035e-43  0.4906769 0.384 0.205 4.028688e-38
chr19-16368699-16370559 2.574878e-40  0.4655875 0.363 0.203 1.089637e-35
chr10-62183251-62184119 6.249364e-38  0.5604293 0.282 0.112 2.644606e-33
chr13-73984798-73985553 1.554884e-36  0.5434926 0.207 0.073 6.579959e-32
chr8-19496588-19497721  3.010657e-35  0.5070322 0.284 0.130 1.274050e-30
chr18-26215422-26216120 2.460344e-34  0.5215445 0.190 0.065 1.041169e-29
                        p_val_adj_BH
chr17-82126354-82128111 3.731758e-38
chr19-16368699-16370559 5.046632e-36
chr10-62183251-62184119 8.165627e-34
chr13-73984798-73985553 1.523748e-32
chr8-19496588-19497721  2.360295e-31
chr18-26215422-26216120 1.607384e-30
[1] "names.sig"
    [1] "chr17-82126354-82128111"   "chr19-16368699-16370559"  
    [3] "chr10-62183251-62184119"   "chr13-73984798-73985553"  
    [5] "chr8-19496588-19497721"    "chr18-26215422-26216120"  
    [7

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image

Saving 6.67 x 6.67 in image



In [ ]:
all.names.sig 

In [ ]:
gene.maps <- ClosestFeature(se, all.names.sig)
gene.maps <- gene.maps %>% dplyr::mutate(gene.id=paste0(gene_name, "_", type))
rownames(gene.maps) <- all.names.sig
gene.maps

In [ ]:
btwn_embeddings <- file.path(btwnClust.outdir, "embeddings")
dir.create(btwn_embeddings)

featplot <- function(name.sig, se, curr.outdir, maps=NULL){
    feat <- FeaturePlot(se,  features=name.sig)
    name <- name.sig
    if (!is.null(maps)){
        name <- maps[name.sig, "gene.id"]
        feat <- feat + ggtitle(name)
    }
    ggsave(plot=feat,
           file=file.path(curr.outdir, paste0(name,".embedFeat.top3.png")))
    }

sapply(all.names.sig[1:20], FUN=featplot, se=se, curr.outdir=btwn_embeddings, maps=gene.maps)